In [ ]:
# ваш код

# Регрессия

Этот набор данных представляет собой запись о 7 распространенных различных видах рыб, продаваемых на рыбном рынке. С помощью этого набора данных можно обучить модель, которая будет предсказывать **вес рыбы**.

- Weight - weight of fish in Gram g
- Length1 - vertical length in cm
- Length2 - diagonal length in cm
- Length3 - cross length in cm
- Height - height in cm
- Width - diagonal width in cm

## EDA

1) Импортируйте файл Fish.csv, записав в переменную **df**

In [ ]:
# ваш код

2) Вывести первые пять строк

In [ ]:
# ваш код

3) Выведите размер датасета

In [ ]:
# ваш код

4) Вывести общую информацию о датафрейме при помощи метода info()

In [ ]:
# ваш код

5) Вывести основные описательные статистики для числовых и категориальных признаков

In [ ]:
# ваш код

6) Постройте тепловую карту для оценки **корреляции признаков**, что можно сказать на основании графика?

In [ ]:
# ваш код

7) Удалите скоррелированные числовые признаки, где корреляция больше 0.9, при помощи кода ниже, где:
- df - исходный датасет
- to_drop - колонки, которые необходимо удалить

**Результат запишите в переменную df_clean**, переменную df в данном случае оставьте без изменений

In [ ]:
cor_matrix = # ваш код.drop(['Weight'], axis=1).corr().abs()

upper_tri = cor_matrix.where(
    np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))

to_drop = [
    column for column in upper_tri.columns if any(upper_tri[column] > 0.9)
]

In [ ]:
# ваш код

8) Проанализируйте **целевую переменную Weight в df_clean**, что вы можете сказать о распределении? Нормально ли оно? проверьте при помощи стат метода

In [ ]:
# ваш код

9) Если распределение не нормальное, то **прологарифмируйте**  таргет Weight с **добавлением внутри единицы**, **записав значения в новую колонку Weight_log** в переменнной df_clean

Проверьте, стало ли распределение нормальным

In [ ]:
# ваш код

10) Создайте новую переменную **df_label**, присвойте ей трансформированный датасет df_clean при помощи One-hot кодирования (лучше использовать pandas.get_dummies() для бинаризации)

Примените drop_first = True

In [ ]:
# ваш код

## Modeling

11) Разбиение данных на train/test

- Разбейте данные на тренировочные и тестовые test_size=0.25
- Установите константу RAND = 10 в самом начале и используйте далее как random_state
- Не забывайте про стандартизацию
- Не забывайте, что нужно для записи в переменную X удалить целевые переменные 'Weight', 'Weight_log'
- Если вы логарифмировали y_test, то незабудьте создать к примеру новую переменную y_test_exp (произвести потенционирование), чтобы правильно анализировать значения метрик

In [ ]:
# ваш код

12) Необходимо реализовать алгоритм Linear Regression при помощи метода оптимизации **стохастический градиентный спуск**

Необходимо взять пример из лекции, где реализована линейная регрессия без использования sklearn, и немного поменять алгоритм. Не забываем, что нам нет необходимости суммировать результат по всей выборке! А также будьте аккуратны при вычитании антиградиента (не будет нормирования на всю длину X)!

In [ ]:
# ваш код

In [ ]:
def r2_adjusted(y_true: np.ndarray, y_pred: np.ndarray,
                X_test: np.ndarray) -> float:
    """Коэффициент детерминации (множественная регрессия)"""
    N_objects = len(y_true)
    N_features = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)


def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean percentage error"""
    return np.mean((y_true - y_pred) / y_true) * 100


def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean absolute percentage error"""
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100


def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Weighted Absolute Percent Error"""
    return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100


def huber_loss(y_true: np.ndarray, y_pred: np.ndarray, delta: float = 1.345):
    """Функция ошибки Хьюбера"""
    assert len(y_true) == len(y_pred), 'Разные размеры данных'
    huber_sum = 0
    for i in range(len(y_true)):
        if abs(y_true[i] - y_pred[i]) <= delta:
            huber_sum += 0.5 * (y_true[i] - y_pred[i])**2
        else:
            huber_sum += delta * (abs(y_true[i] - y_pred[i]) - 0.5 * delta)
    huber_sum /= len(y_true)
    return huber_sum


def logcosh(y_true: np.ndarray, y_pred: np.ndarray):
    """функция ошибки Лог-Кош"""
    return np.sum(np.log(np.cosh(y_true - y_pred)))


def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    """
    The Root Mean Squared Log Error (RMSLE) metric 
    Логаритмическая ошибка средней квадратичной ошибки
    """
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except:
        return None


def get_metrics(y_test: np.ndarray,
                y_pred: np.ndarray,
                X_test: np.ndarray,
                name: str = None,
                delta: float = 1.345):
    """Генерация таблицы с метриками"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]

    df_metrics['MAE'] = mean_absolute_error(y_test, y_pred)
    df_metrics['MSE'] = mean_squared_error(y_test, y_pred)
    df_metrics['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
    df_metrics['RMSLE'] = rmsle(y_test, y_pred)
    df_metrics['R2 adjusted'] = r2_adjusted(y_test, y_pred, X_test)
    # df_metrics['Huber_loss'] = huber_loss(y_test, y_pred, delta)
    # df_metrics['Logcosh'] = logcosh(y_test, y_pred)
    df_metrics['MPE_%'] = mpe(y_test, y_pred)
    df_metrics['MAPE_%'] = mape(y_test, y_pred)
    df_metrics['WAPE_%'] = wape(y_test, y_pred)

    return df_metrics

13) Обучите разработанную модель линейной регрессии с использованием следующих значений параметров на train выборке:
    
- learning_rate=0.001
- eps=0.0000001
- iters=4000

In [ ]:
# ваш код

14) Какие результаты по основным метрикам вы получили на test? Что можно сказать про модель?

- Не забудьте преобразовать обратно целевую переменную (убрать логарифм) для анализа метрик (для y_test и y_predict), но в обучении участвуют прологарифмированные данные.

- На какую метрику **более релевантно смотреть на основании распределения целевой перменной**?
- На подобие как в лекции, сделайте отдельный вывод метрик в DataFrame и запишите **результат в переменную metrics**

In [ ]:
# ваш код

15) Сформируйте **таблицу DataFrame с расчитанными весами** для каждого из признаков на основании обученного ранее алгоритма:
- где первый столбец **feature** - содержатся названия признаков
- второй столбец **score** - содержатся значения весов перед соответстввующими признаками

In [ ]:
# ваш код

16) Примените стандартный метод для линейной регресии из **sklearn**: обучите модель на train

In [ ]:
# ваш код

17) Какие результаты по основным метрикам вы получили на test?

- Что можно сказать про модель?
- Добавьте значения метрик по результатам работы линейной регрессии из sklearn в общий датасет с метриками **в переменную metrics**

In [ ]:
# ваш код

18) Сформируйте **таблицу DataFrame с расчитанными весами** для каждого из признаков на основании обученного ранее алгоритма:
- где первый столбец **feature** - содержатся названия признаков
- второй столбец **score** - содержатся значения весов перед соответстввующими признаками

In [ ]:
# ваш код

# Борьба с переобучением

19) Примените Ridge, Lasso и ElasticNet (sklearn.linear_model) регуляризации при alpha=0.001, какие результы для каждой модели вы получили на основании предыдущих метрик?

Аналогично добавьте метрики по результатам работы алгоритмов в переменную metrics.

In [ ]:
# ваш код

20) На основании распределения (даже после преобразования) используйте необходимую метрику или неск метрик, и напишите, какая из моделей была лучше

In [ ]:
# ваш ответ

## Кросс-валидация с Holdout

21) Примените **к наилучшей выбранной модели**, которую вы уже обучали, **кросс-валидацию KFold** с 3мя фолдами на holdout (пример в лекции). Брать именно обученную модель не нужно, необходимо внутри кросс-валидации заново создавать объект экземпляра класса модели с параметрами, которые у нее были.

- Не забывайте про ранее зафиксированный **random_state**
- В качестве метрики возьмите среднюю абсолютную ошибку **mean_absolute_error**
- На каждом фолде подсчитать **значение MAE на validation** данных и **вывести**
- Не забывать про обратное преобразование целевой переменной из логарифма (**потенционирование**)
- Выведите по итогу **среднее значение MAE** полученное на всех фолдах

In [ ]:
# ваш код

22) Получите **предсказанные значения на Holdout** данных при помощи усреднения значений, полученных при обучении на фолдах

**Сравните** результат MAE на validation (OOF) и на test данных (Holdout)


Подсказка:
- для получения средних значений на Holdout используйте np.mean(np.column_stack(****), axis=1)

In [ ]:
# ваш код

23) Добавьте в датасет с метриками **metrics** результаты обучения модели с **KFold на Holdout данных**.

Где вы получили самые наилучшие результаты?

In [ ]:
# ваш код

# Классификация

Компания, которая активно занимается Big Data и Data Science, хочет нанять специалистов по анализу данных среди людей, которые успешно проходят ее курсы. Многие люди регистрируются на обучение и компания хочет понимать, кто из этих кандидатов действительно хочет работать в компании после обучения или поиска новой работы, потому что это помогает снизить стоимость и время, а также повысить качество обучения, оптимизировать расписание курсов, понимать портрет кандидата. 


- enrolle_id - Уникальный идентификатор кандидата
- city - Код города
- city_ development _index - Индекс развития города (масштабированный)
- gender - пол кандидата
- relevent_experience - релевантный опыт кандидата
- enrolled_university - тип обучения в университете (если был)
- education_level - уровень образования кандидата
- major_discipline - основная специальность по образованию
- experience - общий стаж кандидата в годах
- company_size - размер компании в работниках
- company_type - тип работодателя
- lastnewjob - дельта в годах между предыдущей и текущей работах
- training_hours - кол-во завершенных часов обучения
- target 0 – Не ищу новую работу
- target 1 – Ищу новую работу



Спрогнозировать вероятность того, что кандидат будет искать новую работу или будет работать в компании (0 - не ищу новую работу). target - целевая переменная

## EDA

24) Импортируйте файл aug_train.csv, записав в переменную **df**

In [ ]:
# ваш код

25) Вывести первые пять строк

In [ ]:
# ваш код

26) Выведите размер датасета

In [ ]:
# ваш код

27) Вывести общую информацию о датафрейме при помощи метода info()

In [ ]:
# ваш код

28) Вывести основные описательные статистики для числовых и категориальных признаков

In [ ]:
# ваш код

29) Проанализировать кол-во пропусков, а также посмотреть сколько это в процентах от размера датасета (кол-ва строк)

In [ ]:
# ваш код

30) Заполните пропусками датасет

In [ ]:
# ваш код

31) Посмотрите на целевую переменную **target** при помощи seaborn.countplot. Какое соотношение классов? Есть ли дисбаланс?

In [ ]:
# ваш код

32) Создайте новую переменную **df_label**, присвойте ей трансформированный датасет df при помощи One-hot кодирования (лучше использовать pandas.get_dummies() для бинаризации)

- Примените drop_first = True
- Не забудьте удалить индексы ('enrollee_id')

In [ ]:
# ваш код

In [ ]:
def get_metrics(y_test, y_pred, y_score, name):
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]

    # Основные метрики для задачи классификации
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
    df_metrics['Precision'] = precision_score(y_test, y_pred)
    df_metrics['Recall'] = recall_score(y_test, y_pred)
    df_metrics['f1'] = f1_score(y_test, y_pred)
    df_metrics['Logloss'] = log_loss(y_test, y_score)

    return df_metrics

## Modeling Baseline

33) Разбиение данных на train/test

- Разбейте данные на тренировочные и тестовые test_size=0.25
- Установите константу RAND = 10 в самом начале и используйте далее как random_state
- Не забывайте про стандартизацию
- Установите stratify=y
- Не забывайте, что нужно для записи данных в переменную X удалить целевую переменную target

In [ ]:
# ваш код

34) Примените стандартный метод для логистической регресии из **sklearn**

- Обучите модель на train и получите предсказанные значения на test
- Используйте для baseline параметр **class_weight, И ТОЛЬКО ЕГО**
- Сделайте отдельный вывод метрик в DataFrame и запишите **результат в переменную metrics**

In [ ]:
# ваш код

35) Постройте график кривой ROC-AUC

In [ ]:
# ваш код

## GridSearch

36) Найдите параметры для LogisticRegression, используя StratifiedKFold с 3 фолдами, при помощи **GridSearch**

- Метрика для оптимизации scoring = 'roc_auc'

- Нужно использовать такие параметры как: penalty, C, solver, l1_ratio, tol, class_weight

- Не забудьте для StratifiedKFold зафиксировать random_state

In [ ]:
# ваш код

37) Выведите наилучшие значения метрик и параметры после обучения GridSearchCV

In [ ]:
# ваш код

38) Найдите параметры для LogisticRegression, используя StratifiedKFold с 3 фолдами, при помощи **RandomizedSearchCV**

- Метрика для оптимизации scoring = 'roc_auc'

- Нужно использовать такие параметры как: penalty, C, solver, l1_ratio, tol, class_weight

- Не забудьте для StratifiedKFold зафиксировать random_state

In [ ]:
# ваш код

39) Выведите наилучшие значения метрик и параметры после обучения RandomizedSearchCV

In [ ]:
# ваш код

## Holdout

40) Обучение на фолдах на лучших параметрах

- Далее используйте кросс-валидацию со стратификацией **StratifiedKFold с 3мя фолдами** как делали ранее
- Не забудьте для StratifiedKFold зафиксировать **random_state**
- Подайте на вход модели LogisticRegression() **наилучшие параметры**, полученные путем поиска по сетке (выбрать между параметрами GridSearchCV и RandomizedSearchCV по значениям метрик - атрибут *.best_score_)
- В качестве метрики возьмите **ROC_AUC**
- На каждом фолде подсчитать значение **ROC-AUC на validation** данных и **вывести**
- Выведите по итогу **среднее значение ROC-AUC** полученное на всех фолдах

In [ ]:
# ваш код

41) Получите **предсказанные значения на Holdout** данных при помощи усреднения значений (для вероятностй) и моды (для меток классов), полученных при обучении на фолдах

**Сравните** результат ROC-AUC на validation (OOF) и на test данных (Holdout)


Подсказка:
- для получения средних значений на Holdout используйте np.mean(np.column_stack(****), axis=1)
- для получения моды на Holdout используйте stats.mode(np.column_stack(****), axis=1)[0]

In [ ]:
# ваш код

42) Добавьте в датасет с метриками **metrics** результаты обучения модели с **StratifiedKFold на Holdout данных**.

Где вы получили самые наилучшие результаты?

In [ ]:
# ваш код

Улучшились ли результаты на Holdout по сравнению c Baseline? **Если нет, попробуйте тщательнее подобрать значения в методах поиска по сетке и снова посмотрите результаты.**

# Задание со звездочкой

Найти гиперпараметры для LogisticRegression при помощи **Optuna**, предварительно установив библиотеку. Использовать аналогичный подход с использованием кросс-валидации. Необходимо при помощи Optuna улучить результаты с прошлыми обученными моделями, только в этом случае задание засчитывается.

Пример того, как находить параметры и как получить итоговый результат:

https://github.com/miracl1e6/nyc-taxi-trip-duration/blob/master/nyc-taxi-lgboost-lama.ipynb

Дополнительные материалы, которые могут помочь разобраться:
- https://optuna.org
- https://github.com/optuna/optuna
- https://coderzcolumn.com/tutorials/machine-learning/simple-guide-to-optuna-for-hyperparameters-optimization-tuning

По данному заданию консультация не проводится, вы должны **САМОСТОЯТЕЛЬНО** разобраться, задание по желанию, оно не обязательное. Учтите, что при комбинации некоторых гиперпараметров, могут высвечиваться ошибки, надо подумать, как это обработать

In [ ]:
# ваш код